# Table of Contents

* [4 General Functions (Review)](#general_functions)
    * [4.1 Functions to Save and Open Variables](#open_save)
* [5 Document Clustering](#document_clustering) 
    * [5.1 k-Means Clustering](#k_means)
    * [5.2 Density-based Spatial Clustering of Applications with Noise (DBSCAN)](#dbscan)
    * [5.3 Balanced Iterative Reducing and Clustering using Hierarchies (Birch)](#birch)
    * [5.4 Affinity Propagation](#prop)

# General Functions <a id='general_functions'></a>

## Functions to Save and Open Variables <a id='open_save'></a>

Since it is not uncommon for a machine learning task to take a long time it is good practice to save variables that may be needed in the future. This can be achieved by using the [pickle](https://docs.python.org/3/library/pickle.html) module. This package allows a variable up to 4gb to be saved. This limitation is why the 'metrics' variables are saved as individual items instead of a dictionary.

In [1]:
# Save variables to file
import pickle

def save_var(variable_name):
    """ Saves the variable with the provided variable name 
         in the global namespace to the ./vars folder 
         with the provided same name """
    
    with open('./vars/' + variable_name,'wb') as my_file_obj:
        pickle.dump(globals()[variable_name], my_file_obj, protocol=pickle.HIGHEST_PROTOCOL)

def save_var_list(variable_name_list):
    """ Saves each variable with the provided variable name 
         in the global namespace to the ./vars folder 
         with the provided same name """
    for name in variable_name_list:
        with open('./vars/' + name,'wb') as my_file_obj:
            pickle.dump(globals()[name], my_file_obj, protocol=pickle.HIGHEST_PROTOCOL)

def open_var(file_name):
    """ Returns the variable saved with the provided 
         file name located in the ./vars folder"""
    
    file_object = open('./vars/' + file_name,'rb')  

    loaded_var = pickle.load(file_object)
    
    return loaded_var

def open_var_list(file_name_list):
    """ Loads a variable corresponding to each file name
         in file_name_list to the global namespace. """
    
    for file_name in file_name_list:
        globals()[file_name] = open_var(file_name)

In [2]:
%%time
# Load Datasets
class Dataset_Part:
    """ Represents a dataset with attributes
         data and target """
    
    data = None
    target = None
    def __init__(self, X, y):
        self.data = X
        self.target = y

open_var_list(['mnist_train', 'mnist_test', 'rcv1_train', 'rcv1_test'])

CPU times: user 175 ms, sys: 1.02 s, total: 1.2 s
Wall time: 4.43 s


In [4]:
%%time
open_var_list(['scores_NB_rcv1', 'scores_Mult_NB', 'scores_Gauss_NB', 'scores_Bern_NB','scores_SVM_mnist', 
               'scores_kNN_mnist', 'scores_DT_rcv1', 'scores_DT_mnist'])

In [3]:
%%time
open_var_list(['clf_SVM_mnist', 'pred_SVM_mnist', 'scores_SVM_mnist', 'confidence_SVM_mnist', 'metrics_SVM_mnist'])

# [Document Clustering](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) <a id='document_clustering'></a>

Clustering is an unsupervised training method, meaning it is performed on data without labels. Because of this unsupervised learning is capable of finding relations that may not have been previously observed. 

## [k-Means Clustering](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) <a id='k_means'></a>

This algorithm is implemented in the sklearn.cluster.KMeans scikit-learn module. K-means clustering attempts to seperate data into a predetermined, k, number of clusters. The aim is to create clusters with equal variance, thus minimizing inertia, also known as the within-cluster sum of squares. Inertia is defined as:

\begin{equation*}
 \sum_{i=0}^n \min_{\mu_j \in C} (\mid \mid x_j - \mu_i \mid \mid^2)
\end{equation*}


[comment]: <> (need to reword, too close to source)
To find clusters k-Means has a three step process explained by [Zhao et al.](https://doi.org/10.1016/j.neucom.2018.02.072) [1] as:

1. Initialize k centroids, one for each cluster. The most basic way to do this is by picking k random samples.
+ Assign each sample to the closest centroid.
+ Recompute centroids with assignments from previous step.
+ Repeat step 2 and step 3 until convergence

## [Density-based Spatial Clustering of Applications with Noise (DBSCAN)](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN) <a id='dbscan'></a>

The DBSCAN algorithm clusters samples into areas of high density with surrounding low dennsity areas. Because of this Clusters can be any shape and the number of clusters is not predeturmined. Clusters are formed by finding region that satisfy a minimum density, number of documents per area. The shape of the cluster is determined by the distance metric used. Any distance function can be used and the distance function will determine the shape of the clusters [2].

To form a cluster DBSCAN searches for areas with a minimum number of points within a specified distance, $\varepsilon$, from a central point, this area is called an $\varepsilon$-neighborhood. Each point in a $\varepsilon$-neighborhood will expand outward, and if this neighborhood meets the minimum number of points required the cluster is updated to include this $\varepsilon$-neighborhood. Points that are not within $\varepsilon$ of the center, but it is included in the cluster it is said to be density-reachable. [Good visualization here](https://cse.buffalo.edu/~jing/cse601/fa12/materials/clustering_density.pdf).

## [Balanced Iterative Reducing and Clustering using Hierarchies (Birch)](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.Birch.html) <a id='birch'></a>

[Paper](https://rdcu.be/XdFp)

## [Affinity Propagation](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AffinityPropagation.html#sklearn.cluster.AffinityPropagation) <a id='prop'></a>

Affinity propagation uses exemplars instead of centroids. This means that instead of finding a centroid affinity propagation works by finding samples that are most representative of the other samples. In addition the number of clusters is not predetermined. The number of clusters is determined by the data. 

[comment]: <> (need to reword, too close to source)
There are three main formulas used in affinity propagation.

1. The similarity of two samples is denoted $s(i,k)$.
2. The responsibility of a sample, $k$, to be an exemplar of sample, $i$. 
  
 \begin{equation*} 
   r(i, k) \leftarrow s(i,k) - \max [a(i, k') + s(i, k') \forall k' \neq k] 
 \end{equation*}

3. The accumulated evidence that sample $i$ should choose sample $k$ to be its exemplar.
 
 \begin{equation*}
   a(i, k) \leftarrow \min [0, r(k,k) + \sum_{i' ~ s.t.~ i' \notin \{i,k\}} r(i',k)]
 \end{equation*}

### Bibtex:

#### K means article [1]
@article{ZHAO2018195,
title = "k-means: A revisit",
journal = "Neurocomputing",
volume = "291",
pages = "195 - 206",
year = "2018",
issn = "0925-2312",
doi = "https://doi.org/10.1016/j.neucom.2018.02.072",
url = "http://www.sciencedirect.com/science/article/pii/S092523121830239X",
author = "Wan-Lei Zhao and Cheng-Hao Deng and Chong-Wah Ngo",
keywords = "Clustering, -means, Incremental optimization"
}

#### DBSCAN article [2] 
@inproceedings{Ester:1996:DAD:3001460.3001507,
 author = {Ester, Martin and Kriegel, Hans-Peter and Sander, J\"{o}rg and Xu, Xiaowei},
 title = {A Density-based Algorithm for Discovering Clusters a Density-based Algorithm for Discovering Clusters in Large Spatial Databases with Noise},
 booktitle = {Proceedings of the Second International Conference on Knowledge Discovery and Data Mining},
 series = {KDD'96},
 year = {1996},
 location = {Portland, Oregon},
 pages = {226--231},
 numpages = {6},
 url = {http://dl.acm.org/citation.cfm?id=3001460.3001507},
 acmid = {3001507},
 publisher = {AAAI Press},
 keywords = {arbitrary shape of clusters, clustering algorithms, efficiency on large spatial databases, handling nlj4-275oise}, } 